In [4]:
import sys
import os

project_path = r"D:\python_project\CNN-MNIST"
if project_path not in sys.path:
    sys.path.append(project_path)

os.chdir(project_path)
print(os.getcwd())


D:\python_project\CNN-MNIST


In [5]:
from utils.data_utils import get_mnist_loaders

train_loader, test_loader, device = get_mnist_loaders(
    data_dir='./data/MNIST',  # 指向你手动放置的数据目录
    batch_size=32,
    use_cuda=False
)

images, labels = next(iter(train_loader))
print(images.shape)  # torch.Size([32, 1, 28, 28])
print(labels[:5])


RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(images[0][0], cmap='gray')
plt.title(f"Label: {labels[0].item()}")
plt.axis('off')
plt.show()

In [ ]:
from model.CNN import CNNnet

model = CNNnet().to(device)

In [7]:
from utils.hparams import get_hyperparams
from model.CNN import CNNnet
from model.train import train
from model.test import test
from utils.data_utils import get_mnist_loaders
import torch.optim as optim

# 加载所有超参数
args = get_hyperparams()

# 数据加载器
train_loader, test_loader, device = get_mnist_loaders(
    data_dir='./data/MNIST/MNIST',
    batch_size=args.batch_size,
    test_batch_size=args.test_batch_size,
    seed=args.seed,
    use_cuda=args.use_cuda,
    shuffle=args.shuffle,
    num_workers=args.num_workers,
    pin_memory=args.pin_memory
)

# 初始化模型与优化器
model = CNNnet().to(device)
optimizer_map = {
    'Adam': optim.Adam,
    'AdaGrad': optim.Adagrad,
    'RMSProp': optim.RMSprop
}
optimizer_cls = optimizer_map[args.optimizer]
optimizer = optimizer_cls(model.parameters(), lr=args.learning_rate)

# 训练 & 测试
for epoch in range(1, args.epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval=args.log_interval)
    test(model, device, test_loader)


RuntimeError: Dataset not found. You can use download=True to download it

In [5]:
import os
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from model.CNN import CNNnet
from model.train import train
from model.test import test
from utils.data_utils import get_mnist_loaders  # 强制重新加载模块
from utils.hparams import get_hyperparams

# 优化器和学习率组合
optimizers = ['Adam', 'AdaGrad', 'RMSProp']
learning_rates = [0.1, 0.01, 0.001]

# 创建结果文件夹
os.makedirs("results", exist_ok=True)

# 结果记录字典
result_matrix = {}

for opt_name in optimizers:
    result_matrix[opt_name] = {}
    for lr in learning_rates:
        print(f"\n===== Running {opt_name} with LR={lr} =====")
        args = get_hyperparams(optimizer=opt_name, lr=lr)
        train_loader, test_loader, device = get_mnist_loaders(
            batch_size=args.batch_size,
            test_batch_size=args.test_batch_size,
            seed=args.seed,
            use_cuda=args.use_cuda,
            shuffle=args.shuffle,
            num_workers=args.num_workers,
            pin_memory=args.pin_memory
        )

        model = CNNnet().to(device)
        optimizer_map = {
            'Adam': optim.Adam,
            'AdaGrad': optim.Adagrad,
            'RMSProp': optim.RMSprop
        }
        optimizer = optimizer_map[opt_name](model.parameters(), lr=args.learning_rate)

        train_losses = []
        test_accuracies = []

        for epoch in range(1, args.epochs + 1):
            loss_vals = train(model, device, train_loader, optimizer, epoch, args.log_interval)
            train_losses.append(sum(loss_vals) / len(loss_vals))
            _, acc = test(model, device, test_loader)
            test_accuracies.append(acc)

        # 保存结果
        result_matrix[opt_name][lr] = {
            'train_loss': train_losses,
            'test_acc': test_accuracies
        }

        # === 保存当前实验的曲线图 ===
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 2, 1)
        plt.plot(range(1, len(train_losses)+1), train_losses, marker='o')
        plt.title(f"Train Loss ({opt_name}, lr={lr})")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(range(1, len(test_accuracies)+1), test_accuracies, marker='x')
        plt.title(f"Test Accuracy ({opt_name}, lr={lr})")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy (%)")
        plt.grid(True)

        plt.tight_layout()
        img_name = f"results/{opt_name}_lr{lr}.png".replace('.', '_')
        plt.savefig(img_name)
        plt.close()



===== Running Adam with LR=0.1 =====
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301220	Grad Norm: 1.8177
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.311620	Grad Norm: 0.1538
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.295421	Grad Norm: 0.0678
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.308861	Grad Norm: 0.1372
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.324122	Grad Norm: 0.1248
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.286736	Grad Norm: 0.1616
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.307298	Grad Norm: 0.1088
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.291949	Grad Norm: 0.1351
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.314352	Grad Norm: 0.1027
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.332623	Grad Norm: 0.1488

Test set: Average loss: 2.3076, Accuracy: 1010/10000 (10.10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.290286	Grad Norm: 0.0697
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.255116	Grad Norm: 0.1320
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.284818	Grad Norm: 0.0740
Train Epoch: 2 [19200/6000

In [ ]:
import os
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from model.CNN import CNNnet
from model.train import train
from model.test import test
from utils.data_utils import get_mnist_loaders
from utils.hparams import get_hyperparams

# 学习率列表
learning_rates = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]

# 创建保存文件夹
os.makedirs("results/adam_lr_experiments", exist_ok=True)

for lr in learning_rates:
    print(f"\n===== Running Adam with LR={lr} =====")
    args = get_hyperparams(optimizer='Adam', lr=lr)
    train_loader, test_loader, device = get_mnist_loaders(
        batch_size=args.batch_size,
        test_batch_size=args.test_batch_size,
        seed=args.seed,
        use_cuda=args.use_cuda,
        shuffle=args.shuffle,
        num_workers=args.num_workers,
        pin_memory=args.pin_memory
    )

    model = CNNnet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    avg_grad_norms = []
    test_accuracies = []
    grad_norms_per_epoch = []

    for epoch in range(1, args.epochs + 1):
        grad_norms = train(model, device, train_loader, optimizer, epoch, args.log_interval)
        avg_grad_norms.append(sum(grad_norms) / len(grad_norms))
        grad_norms_per_epoch.append(grad_norms)
        _, acc = test(model, device, test_loader)
        test_accuracies.append(acc)

    # 绘图
    plt.figure(figsize=(15, 4))

    # 1. 每轮平均梯度范数
    plt.subplot(1, 3, 1)
    plt.plot(range(1, args.epochs + 1), avg_grad_norms, marker='o')
    plt.title(f"Avg Gradient Norm per Epoch (lr={lr})")
    plt.xlabel("Epoch")
    plt.ylabel("L2 Norm")
    plt.grid(True)

    # 2. 测试准确率
    plt.subplot(1, 3, 2)
    plt.plot(range(1, args.epochs + 1), test_accuracies, marker='x', color='green')
    plt.title(f"Test Accuracy per Epoch (lr={lr})")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")
    plt.grid(True)

    # 3. 最后一轮每 batch 的梯度范数
    plt.subplot(1, 3, 3)
    plt.plot(grad_norms_per_epoch[-1], marker='.')
    plt.title(f"Gradient Norm per Batch (Last Epoch, lr={lr})")
    plt.xlabel("Batch")
    plt.ylabel("L2 Norm")
    plt.grid(True)

    plt.tight_layout()
    save_path = f"results/adam_lr_experiments/adam_lr_{str(lr).replace('.', '_')}.png"
    plt.savefig(save_path)
    plt.close()



===== Running Adam with LR=0.1 =====
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301220	Grad Norm: 1.8177
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.311620	Grad Norm: 0.1538
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.295421	Grad Norm: 0.0678
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.308861	Grad Norm: 0.1372
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.324122	Grad Norm: 0.1248
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.286736	Grad Norm: 0.1616
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.307297	Grad Norm: 0.1088
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.291949	Grad Norm: 0.1351
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.314351	Grad Norm: 0.1027
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.332623	Grad Norm: 0.1488

Test set: Average loss: 2.3076, Accuracy: 1010/10000 (10.10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.290286	Grad Norm: 0.0697
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.255116	Grad Norm: 0.1320
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.284819	Grad Norm: 0.0740
